# Ch 17 demo: Scheduled blocks

This is a demo which shows how to schedule a task to run on a specific schedule

## Bluesky Setup

In [ ]:
# Load some code called "Client" from the "atproto" library that will help us work with Bluesky
from atproto import Client

(optional) use the fake version of Bluesky atproto, so you don't have to use real Bluesky passwords

In [ ]:
# Note: I don't yet have the fake_atproto setup for this demo yet
# %run fake_apis/fake_atproto.ipynb

In [ ]:
# Login to Bluesky
%run bluesky_keys.py

client = Client(base_url="https://bsky.social")
client.login(handle, password)

## Timed actions
load `schedule` library and `time` library that helps us with scheduling

In [ ]:
# load a library that gives us a scheduling actions
!pip install schedule
import schedule

# load time library so we can write code to 
# check every second if there is a task to run
from time import sleep

Define a function that will be a task to run on a schedule. See more scheduling examples here: [https://schedule.readthedocs.io/en/stable/examples.html](https://schedule.readthedocs.io/en/stable/examples.html)

We will create a function that automatically blocks anyone who has a dm conversation opened with me. Based on [example code here](https://atproto.blue/en/latest/dm.html).

In [ ]:
# Helper function to block a user
from atproto import models
def block_user(user, verbose=True):
    block_record = models.AppBskyGraphBlock.Record(
      subject = user.did, 
      created_at = client.get_current_time_iso()
    )
    uri = client.app.bsky.graph.block.create(client.me.did, block_record).uri
    print("successfully blocked user: " + str(uri))


# Define a function that when run will say that it will post "it's lunchtime!"
def block_those_who_dm_me():
    print("running block task")

    
    # create client proxied to Bluesky Chat service
    dm_client = client.with_bsky_chat_proxy()
    # create shortcut to convo methods
    dm = dm_client.chat.bsky.convo

    # get a list of the dm conversations I have
    convo_list = dm.list_convos()  
    print("processing " + str(len(convo_list.convos)) + " dm conversations")

    # go through all converastins, blocking everyone and leaving the conversations
    for convo in convo_list.convos:
        # block everyone in the conversation
        for member in convo.members:
            # make sure it isn't myself I am blocking
            if(client.me.did != member.did):
                print("blocking " + str(member.handle))
                block_user(member)
            
        # leave the conversation
        dm_client.chat.bsky.convo.leave_convo({"convo_id": convo.id})
    print()

set the schedule for when the task should run

In [ ]:
# schedule the "block_those_who_dm_me" function to run every 5 minutes
schedule.every(5).seconds.do(block_those_who_dm_me)

Run code that will loop forever, every second running any tasks that are ready to run

In [ ]:
# Loop forever, once a second running every task that needs to be run
while True:
    # if any tasks are ready to run, run them
    schedule.run_pending()
    # pause for 1 second before checking again
    sleep(1)